In [226]:
import pandas as pd
import numpy as np
import os
from glob import glob
import re
import io
from galvani import BioLogic

import read_mpr

import pymysql
from sqlalchemy import create_engine

In [202]:
#load data from folder
ds = read_mpr.mpr_pandas('raw_data')

In [167]:
# list paths of data in .mpr file
datafilespath = glob('raw_data/*.mpr') 
datafilespath = [os.path.splitext(val)[0] for val in [os.path.basename(x) for x in datafilespath]]

In [288]:
ds[0]

,freq/Hz,Re(Z)/Ohm,-Im(Z)/Ohm,|Z|/Ohm,Phase(Z)/deg,time/s,Ewe/V,I/mA,Cs/µF,Cp/µF,cycle number,|Ewe|/V,|I|/A,I Range
0,10001.000000,20.116896,672.406189,672.707031,-88.286346,6.338000,0.004219,-0.001763,0.023667,0.023646,0.0,0.003154,0.000005,117
1,6740.000000,576.129150,936.676025,1099.675781,-58.405228,12.608001,0.003156,-0.003312,0.025210,0.018290,0.0,0.005320,0.000005,117
2,4541.999512,780.680786,1205.806519,1436.465088,-57.079586,18.874001,0.009140,0.000473,0.029060,0.020477,0.0,0.007004,0.000005,117
3,3061.000977,1052.983398,1524.199707,1852.554688,-55.361599,25.136001,0.007566,-0.002180,0.034113,0.023092,0.0,0.009089,0.000005,117
4,2063.000488,1414.236328,1897.714111,2366.724121,-53.305363,31.398001,0.012290,0.000498,0.040653,0.026137,0.0,0.011646,0.000005,117
5,1389.998901,1880.165649,2323.925049,2989.255859,-51.025497,37.660002,0.004534,0.003575,0.049270,0.029778,0.0,0.014743,0.000005,117
6,936.062988,2468.635498,2800.162109,3732.970459,-48.600452,43.948002,0.038119,-0.001907,0.060720,0.034166,0.0,0.018457,0.000005,117
7,630.738159,3192.447510,3317.359619,4603.976074,-46.099274,49.158002,0.002959,0.002490,0.076064,0.039491,0.0,0.020850,0.000005,117
8,425.148743,4069.376709,3866.302979,5613.209961,-43.534122,55.432003,0.007605,0.003457,0.096824,0.045936,0.0,0.027742,0.000005,117
9,286.567932,5113.742676,4430.503418,6766.071777,-40.905380,60.658003,0.044813,-0.001066,0.125354,0.053749,0.0,0.031405,0.000005,117


In [286]:
##struction 1
dbhost='127.0.0.1'
dbport=3306
dbuser='root'
dbpassword='yaoyue0320'
connection_name='EIS'

Connection = pymysql.connect(host=dbhost,
                          port=dbport,
                          user=dbuser,
                          password =dbpassword)
print('connection succeed')


cursor = Connection.cursor()
cursor.execute('DROP DATABASE IF EXISTS cell_configuration;')
cursor.execute('CREATE DATABASE cell_configuration;')

##cell configuration
engine =create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=dbhost, db='cell_configuration', user=dbuser, pw=dbpassword))
filename_read(datafilespath).to_sql('cell_configuration', engine, if_exists='fail',index=False)
print('table cell_configuration created')

Cell_ID = np.unique([x.split('_')[0] for x in datafilespath])
Unique_Index = np.append(np.unique([x.split('_')[0] for x in datafilespath],return_index=True)[1],len(ds))
Tech_Type = [x.split('_')[-2].lower() for x in datafilespath]

##labor data
for i in range(len(Cell_ID)):
    cursor.execute('DROP DATABASE IF EXISTS '+Cell_ID[i])
    cursor.execute('CREATE DATABASE '+Cell_ID[i])
    engine =create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=dbhost, db=Cell_ID[i], user=dbuser, pw=dbpassword))
    step_ID = [x.split('_')[-3] for x in datafilespath]
    for j in range(Unique_Index[i],Unique_Index[i+1]):
        ds[j].to_sql('step'+step_ID[j]+'_'+Tech_Type[j], engine, if_exists='fail',index=False)
        print('table '+'step'+step_ID[j]+'_'+Tech_Type[j]+' of '+Cell_ID[i]+' created')
        
print('All data loaded')


connection succeed
table cell_configuration created
table step01_geis of Cell001 created
table step02_gcpl of Cell001 created
table step03_geis of Cell001 created
table step04_gcpl of Cell001 created
table step01_geis of Cell002 created
table step02_gcpl of Cell002 created
table step03_geis of Cell002 created
table step04_gcpl of Cell002 created
table step01_geis of Cell003 created
table step02_gcpl of Cell003 created
table step03_geis of Cell003 created
table step04_gcpl of Cell003 created
table step01_geis of Cell004 created
table step02_gcpl of Cell004 created
table step03_geis of Cell004 created
table step04_gcpl of Cell004 created
table step01_geis of Cell005 created
table step02_gcpl of Cell005 created
table step03_geis of Cell005 created
table step04_gcpl of Cell005 created
table step01_geis of Cell006 created
table step02_gcpl of Cell006 created
table step03_geis of Cell006 created
table step04_gcpl of Cell006 created
table step01_geis of Cell007 created
table step02_gcpl of Ce

In [362]:
#struction 2
dbhost='127.0.0.1'
dbport=3306
dbuser='root'
dbpassword='yaoyue0320'
connection_name='EIS'

Connection = pymysql.connect(host=dbhost,
                          port=dbport,
                          user=dbuser,
                          password =dbpassword)
print('connection succeed')


cursor = Connection.cursor()
cursor.execute('DROP DATABASE IF EXISTS cell_configuration;')
cursor.execute('CREATE DATABASE cell_configuration;')

##cell configuration
engine =create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=dbhost, db='cell_configuration', user=dbuser, pw=dbpassword))
filename_read(datafilespath).to_sql('cell_configuration', engine, if_exists='fail',index=False)
print('table cell_configuration created')

Cell_ID = np.unique([x.split('_')[0] for x in datafilespath])
Unique_Index = np.append(np.unique([x.split('_')[0] for x in datafilespath],return_index=True)[1],len(ds))
Tech_Type = [x.split('_')[-2].lower() for x in datafilespath]

##labor data
for i in range(len(Cell_ID)):
    cursor.execute('DROP DATABASE IF EXISTS '+Cell_ID[i])
    cursor.execute('CREATE DATABASE '+Cell_ID[i])
    engine =create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=dbhost, db=Cell_ID[i], user=dbuser, pw=dbpassword))
    step_ID = [x.split('_')[-3] for x in datafilespath]
    
    geis_data_index=[]
    for j in range(Unique_Index[i],Unique_Index[i+1]):
        if j in np.where(np.array(Tech_Type)== 'geis')[0]:
            geis_data_index = np.append(geis_data_index,j)
    geis_data = pd.concat([ds[int(k)] for k in geis_data_index],ignore_index=True)
    geis_data.to_sql('geis', engine, if_exists='fail',index=False)
    print('geis data of '+Cell_ID[i]+ ' loaded')
    
    gcpl_data_index=[]
    for j in range(Unique_Index[i],Unique_Index[i+1]):
        if j in np.where(np.array(Tech_Type)== 'gcpl')[0]:
            gcpl_data_index = np.append(gcpl_data_index,j)
    gcpl_data = pd.concat([ds[int(k)] for k in gcpl_data_index],ignore_index=True)
    gcpl_data.to_sql('gcpl', engine, if_exists='fail',index=False)    
    print('gcpl data of '+Cell_ID[i]+ ' loaded')    

        
print('All data loaded')


connection succeed
table cell_configuration created
geis data of Cell001 loaded
gcpl data of Cell001 loaded
geis data of Cell002 loaded
gcpl data of Cell002 loaded
geis data of Cell003 loaded
gcpl data of Cell003 loaded
geis data of Cell004 loaded
gcpl data of Cell004 loaded
geis data of Cell005 loaded
gcpl data of Cell005 loaded
geis data of Cell006 loaded
gcpl data of Cell006 loaded
geis data of Cell007 loaded
gcpl data of Cell007 loaded
geis data of Cell008 loaded
gcpl data of Cell008 loaded
All data loaded


In [190]:
def filename_read(path):
    
    index  = ['Cell Nr',
              'Phase Type',
              'Current/μA',
              'Temperature/℃',
              'Relative Humidity/%',
              'Cycle Nr',
              'Technique Step',
              'Technique Type',
              'Channel Nr'
                ]
    frame = pd.DataFrame(columns = index)
    for i in path:
        info = i.split('_')
        value = []
        value.append(re.sub('\D','',info[0]))#cell Nr
        value.append(name[1])#Phase Type
        value.append(re.sub('\D','',info[2]))#current
        value.append(re.sub('\D','',info[3]))#Temperature
        value.append(re.sub('\D','',info[4]))#Relative Humidity
        value.append(re.sub('Cycle','',info[5]))#Cycle number
        value.append(info[6])#Technique Step
        value.append(info[7])#Technique Type
        value.append(re.sub('\D','',info[8]))#Channel Nr
        frame.loc[len(frame)] = value
    
    return frame